In [40]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json

In [19]:
root_path = !pwd
root_path = str(root_path[0])

In [41]:
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

In [42]:
fpath_name = f"{filepath}{filename}"
print(fpath_name)

/home/ernek/Main/Erdos/song_recommender/playlist_data/sampledata/mpd.slice.0-999.json


In [127]:
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

print("File keys: ", data.keys())
print(list(data['playlists'][0].keys()))
print(data['playlists'][0]['tracks'][0].keys())
# df = pd.json_normalize(data, 'locations', ['date', 'number', 'name'], 
#                     record_prefix='locations_')
# print(df)
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

keys = list(keys)
keys.remove('tracks')
print(keys)


File keys:  dict_keys(['info', 'playlists'])
['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms', 'num_artists']
dict_keys(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name'])
['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [396]:
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'],record_prefix = 'track_')

In [397]:
music_df[music_df['playlist_pid'] == 0].columns

Index(['track_pos', 'track_artist_name', 'track_track_uri', 'track_artist_uri',
       'track_track_name', 'track_album_uri', 'track_duration_ms',
       'track_album_name', 'playlist_name', 'playlist_collaborative',
       'playlist_pid', 'playlist_modified_at', 'playlist_num_tracks',
       'playlist_num_albums', 'playlist_num_followers', 'playlist_num_edits',
       'playlist_duration_ms', 'playlist_num_artists', 'playlist_description'],
      dtype='object')

In [398]:
music_df[music_df['playlist_pid'] == 0][['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists', 'playlist_description']]

,track_artist_name,track_track_name,playlist_name,playlist_num_tracks,playlist_num_albums,playlist_duration_ms,playlist_num_artists,playlist_description
0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),Throwbacks,52,47,11532414,37,NaN
1,Britney Spears,Toxic,Throwbacks,52,47,11532414,37,NaN
2,Beyoncé,Crazy In Love,Throwbacks,52,47,11532414,37,NaN
3,Justin Timberlake,Rock Your Body,Throwbacks,52,47,11532414,37,NaN
4,Shaggy,It Wasn't Me,Throwbacks,52,47,11532414,37,NaN
5,Usher,Yeah!,Throwbacks,52,47,11532414,37,NaN
6,Usher,My Boo,Throwbacks,52,47,11532414,37,NaN
7,The Pussycat Dolls,Buttons,Throwbacks,52,47,11532414,37,NaN
8,Destiny's Child,Say My Name,Throwbacks,52,47,11532414,37,NaN
9,OutKast,Hey Ya! - Radio Mix / Club Mix,Throwbacks,52,47,11532414,37,NaN


In [399]:
# music_df[music_df['playlist_num_tracks'] == 20]['playlist_pid'].unique()

In [416]:
Ntracks = 20 
# print(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_artist_name'].unique())
subset_df = music_df[music_df['playlist_num_tracks'] == Ntracks]
print('Number of records: ', len(subset_df))
print('Number of unique artist, song pairs: ', len(subset_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique()))
print('Number of unique artists: ', len(subset_df['track_artist_name'].unique()))

Number of records:  200
Number of unique artist, song pairs:  197
Number of unique artists:  157


In [417]:
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists']

In [418]:
artist_count_df = subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort=False).value_counts(sort=False).reset_index(name="artist_count")

In [419]:
artist_count_df

,playlist_name,track_artist_name,artist_count
0,✔️,Avenged Sevenfold,7
1,✔️,Deftones,2
2,✔️,Killswitch Engage,4
3,✔️,Slipknot,3
4,✔️,Chevelle,1
...,...,...,...
161,playlist,Sia,1
162,playlist,Alessia Cara,1
163,playlist,Lady Gaga,1
164,playlist,Drake,1


In [420]:
subset_df[['playlist_name', 'track_artist_name']]

,playlist_name,track_artist_name
0,✔️,Avenged Sevenfold
1,✔️,Deftones
2,✔️,Killswitch Engage
3,✔️,Avenged Sevenfold
4,✔️,Slipknot
...,...,...
195,playlist,Alessia Cara
196,playlist,Coldplay
197,playlist,Lady Gaga
198,playlist,Drake


In [421]:
subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort = False).value_counts(sort=False).reset_index(name="artist_count").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   playlist_name      166 non-null    object
 1   track_artist_name  166 non-null    object
 2   artist_count       166 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.0+ KB


In [422]:
music_df = pd.merge(subset_df, artist_count_df,  how='left', left_on = ['playlist_name','track_artist_name'], right_on = ['playlist_name','track_artist_name'])

In [423]:
music_df

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,...,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description,artist_count_x,artist_count_y
0,0,Avenged Sevenfold,spotify:track:5o8OdhjVZSF2wCj7M8UpMN,spotify:artist:0nmQIMXWTXfhgOBdNzhGOs,Natural Born Killer,spotify:album:7vfuTRXIAYJz5Uc8SddnTr,315173,Nightmare,✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,7,7
1,1,Deftones,spotify:track:1FdU4oF88UOJwA9GYgognx,spotify:artist:6Ghvu1VvMGScGpOUJBAHNH,Diamond Eyes,spotify:album:4RQnFSkkZlA65Xxchhnaha,188266,Diamond Eyes,✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,2,2
2,2,Killswitch Engage,spotify:track:6dLXK2ppW3GiNEHN2vcMWl,spotify:artist:37394IP6uhnjIpsawpMu4l,Rose Of Sharyn,spotify:album:6NrMjaGIZAKZLMzVnkNY4V,216306,The End Of Heartache [Special Edition],✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,4,4
3,3,Avenged Sevenfold,spotify:track:69weHQOQY0jTUisIo13F7y,spotify:artist:0nmQIMXWTXfhgOBdNzhGOs,I Won't See You Tonight Part 2,spotify:album:4A6vIZBEKEAGoBsaWAZ0ck,284693,Waking The Fallen,✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,7,7
4,4,Slipknot,spotify:track:6wqJeItl3Vc3az4ZicSQAB,spotify:artist:05fG473iIaoy82BF1aGhL8,Before I Forget,spotify:album:4ZDBQSIDIZRUBOG2OHcN3T,278600,Vol. 3: The Subliminal Verses [Special Package],✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,15,Alessia Cara,spotify:track:0prNGof3XqfTvNDxHonvdK,spotify:artist:2wUjUUtkb5lvLKcGKsKqsR,Scars To Your Beautiful,spotify:album:3rDbA12I5duZnlwakqDdZa,230226,Know-It-All,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,1,1
196,16,Coldplay,spotify:track:69uxyAqqPIsUyTO8txoP2M,spotify:artist:4gzpq5DPGxSnKTe4SA8HAU,Adventure Of A Lifetime,spotify:album:3cfAM8b8KqJRoIzt3zLKqw,263786,A Head Full Of Dreams,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,2,2
197,17,Lady Gaga,spotify:track:5ou59ERA2f7ffyAxAcZyVz,spotify:artist:1HY2Jd0NmPuamShAr6KMms,Perfect Illusion,spotify:album:4JiY4JUvXdEA7UFIbiAyor,182000,Joanne,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,1,1
198,18,Drake,spotify:track:1xznGGDReH1oQq0xzbwXa3,spotify:artist:3TVXtAsR1Inumwj472S9r4,One Dance,spotify:album:3hARKC8cinq3mZLLAEaBh9,173986,Views,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,1,1


In [424]:
# subset_df.join(.sum(), on='playlist_name', rsuffix='_count')
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

track_artist_name,21 Savage,Alanis Morissette,Alessia Cara,Alex Aiono,Aminé,Amy Adams,Amy Winehouse,Anika Noni Rose,Anne-Marie,Avenged Sevenfold,...,Tiny Tim w/ The New Duncan Imperials,Tom Jones,Toploader,Toto,Trippie Redd,Van Halen,Whitney Houston,Yeah Yeah Yeahs,Zara Larsson,a-ha
playlist_name,,,,,,,,,,,,,,,,,,,,,
80s,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,1
HALLOWEEN,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
October 2017,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
no,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
party,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
playlist,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
songs to sing in the shower,0,0,0,0,0,0,1,0,0,0,...,0,1,1,1,0,0,1,0,0,0
three,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
✔️,0,0,0,0,0,0,0,0,0,7,...,0,0,0,0,0,0,0,0,0,0


In [425]:
music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1)

0                 Avenged Sevenfold,.Natural Born Killer
1                                 Deftones,.Diamond Eyes
2                      Killswitch Engage,.Rose Of Sharyn
3      Avenged Sevenfold,.I Won't See You Tonight Part 2
4                              Slipknot,.Before I Forget
                             ...                        
195                Alessia Cara,.Scars To Your Beautiful
196                    Coldplay,.Adventure Of A Lifetime
197                          Lady Gaga,.Perfect Illusion
198                                     Drake,.One Dance
199                                    FRENSHIP,.Capsize
Length: 200, dtype: object

In [426]:
music_df[music_df['playlist_num_tracks'] == Ntracks]

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,...,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description,artist_count_x,artist_count_y
0,0,Avenged Sevenfold,spotify:track:5o8OdhjVZSF2wCj7M8UpMN,spotify:artist:0nmQIMXWTXfhgOBdNzhGOs,Natural Born Killer,spotify:album:7vfuTRXIAYJz5Uc8SddnTr,315173,Nightmare,✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,7,7
1,1,Deftones,spotify:track:1FdU4oF88UOJwA9GYgognx,spotify:artist:6Ghvu1VvMGScGpOUJBAHNH,Diamond Eyes,spotify:album:4RQnFSkkZlA65Xxchhnaha,188266,Diamond Eyes,✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,2,2
2,2,Killswitch Engage,spotify:track:6dLXK2ppW3GiNEHN2vcMWl,spotify:artist:37394IP6uhnjIpsawpMu4l,Rose Of Sharyn,spotify:album:6NrMjaGIZAKZLMzVnkNY4V,216306,The End Of Heartache [Special Edition],✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,4,4
3,3,Avenged Sevenfold,spotify:track:69weHQOQY0jTUisIo13F7y,spotify:artist:0nmQIMXWTXfhgOBdNzhGOs,I Won't See You Tonight Part 2,spotify:album:4A6vIZBEKEAGoBsaWAZ0ck,284693,Waking The Fallen,✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,7,7
4,4,Slipknot,spotify:track:6wqJeItl3Vc3az4ZicSQAB,spotify:artist:05fG473iIaoy82BF1aGhL8,Before I Forget,spotify:album:4ZDBQSIDIZRUBOG2OHcN3T,278600,Vol. 3: The Subliminal Verses [Special Package],✔️,false,...,1506470400,20,16,1,4,5843925,7,NaN,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,15,Alessia Cara,spotify:track:0prNGof3XqfTvNDxHonvdK,spotify:artist:2wUjUUtkb5lvLKcGKsKqsR,Scars To Your Beautiful,spotify:album:3rDbA12I5duZnlwakqDdZa,230226,Know-It-All,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,1,1
196,16,Coldplay,spotify:track:69uxyAqqPIsUyTO8txoP2M,spotify:artist:4gzpq5DPGxSnKTe4SA8HAU,Adventure Of A Lifetime,spotify:album:3cfAM8b8KqJRoIzt3zLKqw,263786,A Head Full Of Dreams,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,2,2
197,17,Lady Gaga,spotify:track:5ou59ERA2f7ffyAxAcZyVz,spotify:artist:1HY2Jd0NmPuamShAr6KMms,Perfect Illusion,spotify:album:4JiY4JUvXdEA7UFIbiAyor,182000,Joanne,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,1,1
198,18,Drake,spotify:track:1xznGGDReH1oQq0xzbwXa3,spotify:artist:3TVXtAsR1Inumwj472S9r4,One Dance,spotify:album:3hARKC8cinq3mZLLAEaBh9,173986,Views,playlist,false,...,1474848000,20,19,1,8,4413536,14,NaN,1,1


In [427]:
music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).value_counts() 

Rick Springfield,.Jessie's Girl                         2
PnB Rock,.Selfish                                       2
Toto,.Africa                                            2
Avenged Sevenfold,.Natural Born Killer                  1
Phil Collins,.You'll Be In My Heart                     1
                                                       ..
Dexys Midnight Runners,.Come On Eileen                  1
Billy Joel,.Piano Man                                   1
Phil Collins,.You Can't Hurry Love - 2016 Remastered    1
Amy Winehouse,.Me & Mr Jones                            1
FRENSHIP,.Capsize                                       1
Length: 197, dtype: int64

In [428]:
vc = music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x), axis=1).value_counts()
vc[vc > 1].sum() - len(vc[vc > 1])

3

In [429]:
# for i in df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique():
# #     print(i.split(',.'))
#     if len(i.split(',.')) == 2:
#         print(i)

In [430]:
np.unique(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_track_name'])

array(["(Don't Fear) The Reaper", '1-800-273-8255', '1979', '7/11',
       'Abracadabra', 'Adventure Of A Lifetime', 'Africa', 'Alarm',
       'All Night Long (All Night) - Single Version', 'Almost There',
       'Always',
       'Always Look On The Bright Side of Life - Company Bow - Original Broadway Cast Recording: "Spamalot"',
       'And All Things Will End', 'At The End Of The Day',
       'Bad Bitch (feat. Ty Dolla $ign)', 'Beast and the Harlot',
       'Before I Forget', 'Billie Jean', 'Black Magic Woman',
       "Brandy (You're A Fine Girl)", 'Breathe',
       'Broccoli (feat. Lil Yachty)', "Can't Fight This Feeling",
       'Capsize', 'Caroline', 'Celebrity Skin', 'Cell Block Tango',
       'Chill Bill', 'Closer', 'Come On Eileen', 'Congratulations',
       'Dancing In The Dark', 'Dancing in the Moonlight',
       'December, 1963 (Oh What A Night!)',
       'Defying Gravity - From "Wicked" Original Broadway Cast Recording/2003',
       'Despacito - Remix', 'Desperado', 'Diamo

In [431]:
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists']

In [412]:
music_df[music_df['playlist_num_tracks'] == 20][['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists', 'playlist_description']]

,track_artist_name,track_track_name,playlist_name,playlist_num_tracks,playlist_num_albums,playlist_duration_ms,playlist_num_artists,playlist_description
0,Avenged Sevenfold,Natural Born Killer,✔️,20,16,5843925,7,NaN
1,Deftones,Diamond Eyes,✔️,20,16,5843925,7,NaN
2,Killswitch Engage,Rose Of Sharyn,✔️,20,16,5843925,7,NaN
3,Avenged Sevenfold,I Won't See You Tonight Part 2,✔️,20,16,5843925,7,NaN
4,Slipknot,Before I Forget,✔️,20,16,5843925,7,NaN
...,...,...,...,...,...,...,...,...
195,Alessia Cara,Scars To Your Beautiful,playlist,20,19,4413536,14,NaN
196,Coldplay,Adventure Of A Lifetime,playlist,20,19,4413536,14,NaN
197,Lady Gaga,Perfect Illusion,playlist,20,19,4413536,14,NaN
198,Drake,One Dance,playlist,20,19,4413536,14,NaN


In [413]:
music_df[music_df['playlist_num_tracks'] == Ntracks][important_features].groupby('playlist_name')['track_artist_name'].value_counts()

playlist_name  track_artist_name
80s            Fleetwood Mac        2
               Men At Work          2
               Tears For Fears      2
               Toto                 2
               Bruce Springsteen    1
                                   ..
😴😴             Noah Cyrus           1
               Petit Biscuit        1
               Phora                1
               SZA                  1
               Zara Larsson         1
Name: track_artist_name, Length: 166, dtype: int64